# Динамическое программирование (Dynamic programming)

## Введение

Основная идея метода динамического программирования состоит в том, что если наша задача разбивается на **перекрывающиеся** между собой подзадачи, мы можем не обсчитывать их каждый раз, а сохранять куда-нибудь.

Для примера рассмотрим наивный алгоритм для расчета $n$-того числа Фибоначчи

In [12]:
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1

    return fibonacci(n - 1) + fibonacci(n - 2)

fibonacci(10)

55

Асимптотика - $O(2^n)$

Однако заметим, что при вычислении $F(n)$, $F(n - 2)$ вычисляется дважды (при вычислении $F(n)$ и $F(n - 1)$). $F(n - 3)$ вычисляется уже трижды. В общем случае $F(n - i)$ вычисляется $F(i + 1)$ раз, что очень неэффективно. $F(n - i)$ и есть те самые перекрывающиеся подзадачи.

Применим динамическое программирование для оптимизации

In [ ]:
cache = dict()
cache[0] = 0
cache[1] = 1

def fibonacci(n):
    if n in cache:
        return cache[n]

    f_n = fibonacci(n - 1) + fibonacci(n - 2)
    cache[n] = f_n
    return f_n

fibonacci(100)

Асимптотика этого решения уже $O(n)$, потому что каждое число фибоначчи от $0$ до $n$ будет записано в кэш лишь один раз.

Восходящее/нисходящее дп

## Формальная постановка

**Определение:** Дискретная управляемая система (Discrete control system) $\Omega$ - это упорядоченный набор $\langle D, q, F, V(x), f(x, v), s(x, v)\rangle$, где:
1. $D$ - конечное множество возможных состояний
2. $q \in D$ - начальное состояние
3. $F \subset D$ - множество финальных состояний
4. $V(x)$ - конечное множество возможных в состоянии $x$ управлений
5. $f(x, v)$ - функция перехода из состояния $x$ под воздействием управления $v$ в состояние $x' = f(x, v)$
6. $s(x, v)$ - функция стоимости за переход

**Определение:** Последовательность $T = \{x_0, x_1, ... , x_n\}$ называется траекторией системы, если $x_i = f(x^{i - 1}, v_i)$, где $v_i \in V(x_{i - 1})$

**Определение:** Траектория называется полной, если $x_0 = q$, а $x_n \in F$

**Определение:** Введем отношение $P(x, y)$ : $x$ и $y$ называются достижимыми, если существует траектория из $x$ в $y$

Очевидно, что отношение P рефлексивно, т. е. $P(x, x)$ и транзитивно, т. е. $P(x, y), P(y, z) \Rightarrow P(x, z)$

Так же, для простоты будем рассматривать только дискретные управляемые системы, на которых отношение $P(x, y)$ будет еще и антисимметричным, т. е. $P(x, y), P(y, x) \Rightarrow x = y$. Это означает, что в траекториях между состояниями системы не будет циклов

**Определение:** Функция стоимости траектории $C(T) = \sum_{i = 0}^{n} s(x_i, v_i)$

**Определение:** Функция Беллмана - $V(x_t) = \min_{v_t \in V(x_t)} s(x_t, v_t) + V(x_{t + 1})$

**Принцип:** Беллмана (Bellman principle)
Если из оптимальности траектории $T = \{x_0, x_1, ..., x_n\}$ относительно функции стоимости $С(T)$ следует оптимальность траектории $T(x_i) = \{x_i, ..., x_n\}$, то к задаче оптимизации управляющей системы можно применить метод динамического программирования

Графовое представление

## Задача разрезания стержня

Формулировка задачи:

Имеется стержень длиной $n$ и таблица цен $p_i$ за стержень длины $i$. Необходимо найти разрезание стержня, которое принесет максимальную прибыль $r(n)$. Разрезов может быть любое число.

Уравнение Беллмана для этой задачи:
$r(n) = \max_{1 \leq i \leq n} (p_i + r(n-i))$


In [19]:
def cut_rod(p: list, n: int):
    r = [0] * (n + 1)
    s = [0] * (n + 1)

    for i in range(1, n + 1):
        q = float("-inf")
        for j in range(1, i + 1):
            if q < p[j] + r[i - j]:
                q = p[j] + r[i - j]
                s[i] = j
        r[i] = q

    return r[-1]

cut_rod([0, 1, 5, 8, 9, 10, 17, 17, 20, 24, 30], 10)

30

## Расстояние редактирования

Пусть в пространстве всевозможных слов $L$ определены 3 операции:
1. Замена символа $a$ на $b$
2. Вставка элемента $a$ на позицию $\epsilon$
3. Удаление элемента $a$ с позиции $\epsilon$

И для каждой из них определена цена $w$:
1. $w(a, b)$ - цена замены
2. $w(\epsilon, b)$ - цена вставки
3. $w(a, \epsilon)$ - цена удаления

**Определение:** Расстояние редактирования в пространстве $L$ - это $d(A, B) = \sum_{i = 1}^n w_i$, где $w_n$ - это минимальная по количеству элементов последовательность операций редактирования

Будем рассматривать только те расстояния редактирования $d$, которые являются метриками в пространстве $L$, т.е.:
1. $w(a, a) = 0$
2. $w(a, b) = w(b, a)$
3. $w(a, c) \le w(a, b) + w(b, c)$

**Определение:** Расстояние Левенштейна - расстояние редактирования такое что:
1. $w(a, a) = 0$
2. $w(a, b) = 1$
3. $w(\epsilon, b) = 1$
4. $w(a, \epsilon) = 1$

Уравнение Беллмана для расстояния редактирования:
$d_{A, B}(i, j)= $
1. $0$, $i = j$
2. $i \cdot deleteCost$, $i > 0, j = 0$
3. $j \cdot insertCost$, $i = 0, j > 0$
4. $d(i - 1, j - 1)$, $A_i = B_i$
5. $min(d(i, j - 1) + insertCost, d(i - 1, j) + deleteCost, d(i - 1, j - 1) + replaceCost)$

In [21]:
def redaction_d(A: str, B: str, insertionCost: int = 1, deleteCost: int = 1, replaceCost: int = 1) -> int:
    dp = [None] * (len(B) + 1)
    for i in range(len(B) + 1):
        dp[i] = [0] * (len(A) + 1)
    for i in range(1, (len(B) + 1)):
        dp[i][0] = i * deleteCost
    for i in range(1, (len(A) + 1)):
        dp[0][i] = i * insertionCost

    for i in range(1, (len(B) + 1)):
        for j in range(1, (len(A) + 1)):
            if B[i - 1] == A[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i][j - 1] + insertionCost, dp[i - 1][j] + deleteCost, dp[i - 1][j - 1] + replaceCost)

    return dp[len(B)][len(A)]

redaction_d("wlhauodia", "wwahauobrt")

5

## LCM

In [20]:
def lcs(A: str, B: str) -> int:
    dp = [[0] * (len(A) + 1) for _ in range(len(B) + 1)]

    for i in range(1, len(B) + 1):
        for j in range(1, len(A) + 1):
            if B[i - 1] == A[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
            else:
                dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])

    return dp[len(B)][len(A)]

## Битоническая евклидова задача о коммивояжере

In [23]:
import math

def bitonic_tsp(points):
    points.sort(key=lambda p: (p[0], p[1]))
    n = len(points)
    if n == 0:
        return 0

    dp = [[math.inf] * (n + 1) for _ in range(n + 1)]
    dp[1][1] = 0

    dist = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            dx = points[i][0] - points[j][0]
            dy = points[i][1] - points[j][1]
            dist[i][j] = math.hypot(dx, dy)

    # Заполнение таблицы DP
    for i in range(1, n):
        for j in range(1, i + 1):
            # Обновление правой ветви (i+1 идет вправо)
            dp[i+1][j] = min(dp[i+1][j], dp[i][j] + dist[i-1][i])

            # Обновление левой ветви (i+1 идет влево)
            dp[i+1][i] = min(dp[i+1][i], dp[i][j] + dist[j-1][i])

    # Вычисляем финальный результат, замыкая тур
    result = dp[n][n-1] + dist[n-2][n-1]
    return result

points = [(0, 0), (1, 0), (2, 0)]
print(bitonic_tsp(points))

4.0
